# Multi Layer Perceptron (MLP) 
(c) Deniz Yuret, 2018

* Objectives: See the effect of nonlinearities, learn about regularization and dropout to combat overfitting.
* Prerequisites: Linear models (03.lin.ipynb), AutoGrad, Param, KnetArray, gpu, nll, zeroone
* Knet: xavier, param, param0, relu, dropout (defined and explained)
* Knet: dir, gpu, minibatch, KnetArray (used by mnist.jl)
* Knet: SGD, train!, load, save (used by trainresults)

In [ ]:
# Set display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","Plots"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Knet: Knet, dir, Param, gpu, KnetArray, sgd, progress, nll, zeroone, Data, load, save, gc, relu # param, param0, xavier
using Statistics: mean
using Base.Iterators: flatten

In [ ]:
# Load data (see 02.mnist.ipynb)
include(Knet.dir("data","mnist.jl"))  # Load data
dtrn,dtst = mnistdata(xsize=(784,:)); # dtrn and dtst = [ (x1,y1), (x2,y2), ... ] where xi,yi are minibatches of 100

In [ ]:
# For running experiments
function trainresults(file,model; o...)
    if (print("Train from scratch? "); readline()[1]=='y')
        takeevery(n,itr) = (x for (i,x) in enumerate(itr) if i % n == 1)
        r = ((model(dtrn), model(dtst), zeroone(model,dtrn), zeroone(model,dtst))
             for x in takeevery(length(dtrn), progress(sgd(model,repeat(dtrn,100)))))
        r = reshape(collect(Float64,flatten(r)),(4,:))
        save(file,"results",r)
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        r = Knet.load(file,"results")
    end
    println(minimum(r,dims=2))
    return r
end

In [ ]:
# Some utilities to make model definitions easier:
param(d...; init=xavier, atype=atype())=Param(atype(init(d...)))
param0(d...; atype=atype())=param(d...; init=zeros, atype=atype)
xavier(o,i) = (s = sqrt(2/(i+o)); 2s .* rand(o,i) .- s)
atype()=(gpu() >= 0 ? KnetArray{Float32} : Array{Float32})

## Model definition

In [ ]:
# Redefine Linear model (See lin.ipynb):
struct Linear; w; b; end
Linear(i::Int,o::Int) = Linear(param(o,i),param0(o))
(f::Linear)(x) = (f.w * x .+ f.b)
(f::Linear)(x,y) = nll(f(x),y)
(f::Linear)(d::Data) = mean(f(x,y) for (x,y) in d)

In [ ]:
# Let's define a chain of layers
struct Chain; layers; end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

## Multiple linear layers do not improve over linear model

In [ ]:
# We will first try a chain of multiple linear layers
MultiLinear(h::Int...)=Chain(Linear.(h[1:end-1], h[2:end]))

In [ ]:
# Here is an example model
model=MultiLinear(784,64,10)
println.(summary.((l.w,l.b)) for l in model.layers);

In [ ]:
# Train a linear model
lin1 = trainresults("lin1.jld2", Linear(784,10)); # 113s [0.241861, 0.266827, 0.0668667, 0.0747]

In [ ]:
# Train a multi-linear model
mlp1 = trainresults("mlp1.jld2", MultiLinear(784,64,10)); # 110s [0.242123, 0.285064, 0.0697, 0.0799]

In [ ]:
using Plots; default(fmt=:png,ls=:auto)

In [ ]:
# multilinear converges to a similar solution, not identical because problem is non-convex
plot([lin1[1,:], lin1[2,:], mlp1[1,:], mlp1[2,:]], ylim=(0.0,0.4),
    labels=[:trnLin :tstLin :trnMulti :tstMulti],xlabel="Epochs",ylabel="Loss")  

In [ ]:
# error results also close to the linear model
plot([lin1[3,:], lin1[4,:], mlp1[3,:], mlp1[4,:]], ylim=(0.0,0.1),
    labels=[:trnLin,:tstLin,:trnMulti,:tstMulti], xlabel="Epochs", ylabel="Error")  

## Multiple linear layers are useless because they are equivalent to a single linear layer
If we write down what is being computed and do some algebra, we can show that what is being computed is still an affine function of the input, i.e. stacking multiple linear layers does not increase the representational capacity of the model:

\begin{align*}
\hat{p} &= \mbox{soft}(W_2 (W_1 x + b_1) + b_2) \\
&= \mbox{soft}((W_2 W_1)\, x + W_2 b_1 + b_2) \\
&= \mbox{soft}(W x + b)
\end{align*}

In [ ]:
Knet.gc() # to save gpu memory

## Multi Layer Perceptron (MLP) adds non-linearities between layers

In [ ]:
# Using nonlinearities between layers results in a model with higher capacity and helps underfitting
# relu(x)=max(0,x) is a popular function used for this purpose, it replaces all negative values with zeros.
struct Dense; w; b; f; end
Dense(i::Int,o::Int,f) = Dense(param(o,i),param0(o),f)
(d::Dense)(x) = d.f.(d.w * x .+ d.b)

In [ ]:
function MLP(h::Int...; f=relu)
    mlp = Chain(Dense.(h[1:end-1], h[2:end], f))
    mlp.layers[end].f = identity
    return mlp
end

In [ ]:
function (m::MLP)(x)
    for layer in m.layers
        x = layer(x)
        x = (layer == m.layers[end] ? x : relu.(x))  # <-- This one line makes a big difference
    end
    return x
end

In [ ]:
mlp2 = trainresults("mlp2.jld2", MLP(784,64,10));  # 124s [0.00632429, 0.0888991, 0.00055, 0.0259]

## MLP solves underfitting but still has an overfitting problem

In [ ]:
# MLP solves the underfitting problem!  A more serious overfitting problem remains.
plot([lin1[1,:], lin1[2,:], mlp2[1,:], mlp2[2,:]], ylim=(0.0,0.4),
     labels=[:trnLin :tstLin :trnMLP :tstMLP],xlabel="Epochs",ylabel="Loss")

In [ ]:
# Test error improves from 7.5% to 2.3%!
plot([lin1[3,:], lin1[4,:], mlp2[3,:], mlp2[4,:]], ylim=(0.0,0.1),
    labels=[:trnLin,:tstLin,:trnMLP,:tstMLP], xlabel="Epochs", ylabel="Error")  

In [ ]:
Knet.gc() # to save gpu memory

## MLP with L1 regularization

In [ ]:
# Define new loss functions for L1 and L2 regularization
using Knet: nll
nll1(m::MLP,x,y; λ=0, o...) = nll(m,x,y; o...) + λ * sum(sum(abs, l.w) for l in m.layers)
nll2(m::MLP,x,y; λ=0, o...) = nll(m,x,y; o...) + λ * sum(sum(abs2,l.w) for l in m.layers)

In [ ]:
mlp3 = trainresults("mlp3.jld2", MLP(784,64,10); loss=nll1, λ=4f-5); # 133s [0.0262292, 0.0792939, 0.0066, 0.0237]

In [ ]:
# overfitting less, test loss improves from 0.0808 to 0.0759
plot([mlp2[1,:], mlp2[2,:], mlp3[1,:], mlp3[2,:]], ylim=(0.0,0.15),
     labels=[:trnMLP :tstMLP :trnL1 :tstL1],xlabel="Epochs",ylabel="Loss")

In [ ]:
# however test error does not change significantly: 0.0235 -> 0.0220
plot([mlp2[3,:], mlp2[4,:], mlp3[3,:], mlp3[4,:]], ylim=(0.0,0.04),
     labels=[:trnMLP :tstMLP :trnL1 :tstL1],xlabel="Epochs",ylabel="Error")

In [ ]:
Knet.gc() # to save gpu memory

## MLP with dropout

In [ ]:
using Knet: dropout
@doc dropout

In [ ]:
# Dropout is another way to address overfitting
function (m::MLP)(x; pdrop=0)
    for (i,layer) in enumerate(m.layers)
        p = (i <= length(pdrop) ? pdrop[i] : pdrop[end])
        x = dropout(x, p)     # <-- This one line helps generalization
        x = layer(x)
        x = (layer == m.layers[end] ? x : relu.(x))
    end
    return x
end

In [ ]:
mlp4 = trainresults("mlp4.jld2", MLP(784,64,10); pdrop=(0.2,0.0));  # 119s [0.0126026, 0.0648639, 0.0033, 0.0189]

In [ ]:
# overfitting less, loss results improve 0.0808 -> 0.0639
plot([mlp2[1,:], mlp2[2,:], mlp4[1,:], mlp4[2,:]], ylim=(0.0,0.15),
     labels=[:trnMLP :tstMLP :trnDrop :tstDrop],xlabel="Epochs",ylabel="Loss")

In [ ]:
# this time error also improves 0.0235 -> 0.0188
plot([mlp2[3,:], mlp2[4,:], mlp4[3,:], mlp4[4,:]], ylim=(0.0,0.04),
     labels=[:trnMLP :tstMLP :trnDrop :tstDrop],xlabel="Epochs",ylabel="Error")

In [ ]:
println([:mlperr,minimum(mlp2[4,:]),:L1err,minimum(mlp3[4,:]),:dropouterr,minimum(mlp4[4,:])])

In [ ]:
println([:mlploss,minimum(mlp2[2,:]),:L1loss,minimum(mlp3[2,:]),:dropoutloss,minimum(mlp4[2,:])])

In [ ]:
Knet.gc() # to save gpu memory

## MLP with larger hidden layer and dropout

In [ ]:
# The current trend is to use models with higher capacity tempered with dropout
mlp = trainresults("mlp.jld2", MLP(784,256,10); pdrop=(0.2,0.0));  # 123s [0.00365709, 0.0473298, 0.000283333, 0.0141]

In [ ]:
plot([mlp4[1,:], mlp4[2,:], mlp[1,:], mlp[2,:]],ylim=(0,0.15),
    labels=[:trn64 :tst64 :trn256 :tst256],xlabel="Epochs",ylabel="Loss")

In [ ]:
# We are down to 0.015 error.
plot([mlp4[3,:], mlp4[4,:], mlp[3,:], mlp[4,:]],ylim=(0,0.04),
    labels=[:trn64 :tst64 :trn256 :tst256],xlabel="Epochs",ylabel="Error")